In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd

import tensorflow as tf

%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
label = train['label']

# dense_to_one_hot
Y = pd.get_dummies(label)
X = train.drop('label', axis=1) / 255 # normalize
X = X.astype(np.float32)

# 訓練データと検証データに分割
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y)

In [4]:
# ノードの作成
x = tf.placeholder(tf.float32, shape=(None, train_X.shape[1]))
y_ = tf.placeholder(tf.float32, shape=(None, 10))

In [5]:
# 重みパラメータを作る
def weight_variable(shape):
    init = tf.truncated_normal(shape, stddev=0.1) # std=0.1 regular expression で初期化
    return tf.Variable(init)

# バイアス
def bias_variable(shape):
    init = tf.constant(0.1, shape=shape) # 0.1固定
    return tf.Variable(init)

In [6]:
# CNN

# Convolutin layer
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# pool 2x2 
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
train_X_img = tf.reshape(train_X.values, [-1, 28, 28, 1], )
test_X_img = tf.reshape(test_X.values, [-1, 28, 28, 1])

In [8]:
# Convolution layer 1
W_conv1 = weight_variable([5, 5, 1, 32]) # Filter size 5x5, input_size, output_size
b_conv1 = bias_variable([32]) # ↑の4つ目と同じサイズ

h_conv1 = tf.nn.relu(conv2d(train_X_img, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
# Convolution layer 2
W_conv2 = weight_variable([5, 5, 32, 64]) # 32 -> 64
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
# 接続層
# 7x7の画像 -> 1024のニューロンの完全連結レイヤへ
W_fc = weight_variable([7 * 7 * 64, 1024])
b_fc = bias_variable([1024])

h_pool2flat = tf.reshape(h_pool2, [-1, 7 * 7* 64])
h_fc = tf.nn.relu(tf.matmul(h_pool2flat, W_fc) + b_fc)

In [13]:
# Dropout層 検証中はdropoutしないようにノードをつくって切り替えられるようにする
keep_prob = tf.placeholder(tf.float32) # node
h_fc_drop = tf.nn.dropout(h_fc, keep_prob) # dropout済みのh_fc

# 読み出し層
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc_drop, W_fc2) + b_fc2

In [14]:
# loss
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Adam Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
start = 0
indexes = train_X.index
batch_size = 100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        if start == 0:
            indexes = np.random.permutation(train_X.index)
        end = start + batch_size
        print(start, end)
        di = {x: train_X.loc[indexes[start:end], :], y_: train_Y.loc[indexes[start:end], :], keep_prob: 1.0}
#         if i % 100 == 0:
#             train_accuracy = accuracy.eval(
#                 feed_dict=)
#             print('step %d, training accuracy %g' % (i, train_accuracy))
        
        train_step.run(
            feed_dict={x: train_X.loc[indexes[start:end], :], y_: train_Y.loc[indexes[start:end], :], keep_prob: 0.5})
        
        start = min(end, train_X.shape[0])
            
    
#     print('test accuracy %g' % accuracy.eval(
#         feed_dict={x:test_X_img.eval(), y_:test_Y, keep_prob: 1.0}))

0 100


array([[[[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        ..., 
        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]]],


       [[[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ..., 
         [ 0.],
 